In [1]:
import sys, os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
import h5py as hf
# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
from foundation import sim as SIM
#from foundation.util import replicate, Cloner

np.set_printoptions(linewidth=120)

In [2]:
sim = SIM.Cartpole(timestep=0.03, ctrl_scale=10, 
                   length=0.6, gravity=10., 
                   mass_cart=0.5,
                   fric_cart=0.1, fric_pole=0.1,
                   integration_method='rk4',
                  )
sim

Cartpole(
  (dynamics): Cartpole_Dynamics(
    (controller): Constant()
  )
)

In [3]:
sim.sample_action()

tensor([0.0330])

In [4]:
sim.sample_state()

tensor([0.8173, 4.1547, 0.7732, 2.0094])

In [5]:
u = sim.sample_action()
u = torch.ones(1)*0
u

tensor([0.])

In [6]:
B = None
state0 = torch.tensor([0,np.pi,1,0])
state0 = sim.sample_state(B)

In [7]:
x = sim.reset(state0)

In [8]:
if B is not None and int(np.sqrt(B))**2 == B:
    bb = int(np.sqrt(B))
    fig, axes = plt.subplots(bb,bb)
else:
    fig, ax = plt.subplots(1)

In [10]:
for i in range(200):
    u = sim.sample_action(B)
    x = sim.step(u, N=1)
    imgs = sim.render(200,100)
    if len(imgs.shape) == 4:
        for img, ax in zip(imgs, axes.flat):
            ax.cla()
            ax.imshow(img)
            ax.axis('off')
    else:
        ax.cla()
        ax.imshow(imgs)
        ax.axis('off')
    plt.pause(0.03)